In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
#Importer les fichiers csv comme dataframes 
df_principals = pd.read_csv('title_principals_clean.csv', sep='\t')
df_title_akas = pd.read_csv('title_akas_clean.csv', sep='\t')
df_title_fr = pd.read_csv('title_akas_clean_fr.csv', sep='\t')
df_name_basics = pd.read_csv('name_basics_clean.csv', sep='\t')
df_crew = pd.read_csv('title_crew_clean.csv', sep='\t')
df_title_basics = pd.read_csv('title_basics_clean.csv', sep='\t')
df_ratings = pd.read_csv('title_ratings_clean.csv', sep='\t')
df_tmdb = pd.read_csv('tmdb_clean.csv', sep='\t')
df_french_films = pd.read_csv('data_recommandations_fr.csv', sep='\t', lineterminator='\n')

In [3]:
#df_titres
df_titres = df_title_basics.copy()

In [4]:
#df_titres_1
df_titres_1 = df_titres[['tconst', 'startYear', 'genres']]

In [5]:
#df_titres_fr
df_titres_fr = df_title_fr.copy()

In [6]:
#df_titres_fr_1
df_titres_fr_1 = df_titres_fr[['titleId', 'title']]

In [7]:
#df_langues
#df_langues = df_title_akas.copy()
#df_langues.head()

In [7]:
#df_acteurs
df_acteurs = df_principals.copy()

In [8]:
#df_acteurs_1
df_acteurs_1 = df_acteurs[['tconst', 'nconst', 'category']]

In [9]:
#df_realisateurs
df_realisateurs = df_crew.copy()

In [10]:
#df_equipes
df_equipes = df_name_basics.copy()

In [11]:
#df_equipes_1
df_equipes_1 = df_equipes[['nconst', 'primaryName']]

In [12]:
#df_realisateurs_1
df_realisateurs_1 = pd.merge(df_acteurs_1[df_acteurs_1['category'] == 'director'], df_equipes_1, on='nconst')

In [13]:
#df_acteurs_2
df_acteurs_2 = pd.merge(df_acteurs_1, df_equipes_1, on='nconst')

In [14]:
#df_notes
df_notes = df_ratings.copy()

In [15]:
#df_infos
df_infos = df_tmdb.copy()

In [16]:
#df_infos_1
df_infos_1 = df_infos[['imdb_id', 'original_language', 'overview', 'poster_path']]

In [17]:
#df_films_fr
df_films_fr = df_french_films.copy()

In [18]:
#Fusionner les différents dataframes 
df_merged_1 = pd.merge(df_titres_1, df_titres_fr_1, how="left", left_on='tconst', right_on='titleId')

In [19]:
df_merged_2 = pd.merge(df_merged_1, df_acteurs_2, how="left", left_on='tconst', right_on='tconst')

In [20]:
df_merged_3 = pd.merge(df_merged_2, df_realisateurs_1, how="left", left_on='tconst', right_on='tconst')

In [21]:
df_merged_4 = pd.merge(df_merged_3, df_notes, how="left", left_on='tconst', right_on='tconst')

In [22]:
df_merged_5 = pd.merge(df_merged_4, df_infos_1, how="left", left_on='tconst', right_on='imdb_id')

In [23]:
#Supprimer les colonnes en doublon
df_merged_6 = df_merged_5.drop(['titleId', 'nconst_x', 'nconst_y', 'category_x', 'category_y', 'imdb_id'], axis=1)

In [24]:
#Grouper les acteurs et realisateurs par film
df_merged_7 = df_merged_6.groupby(['tconst', 
                                   'startYear', 
                                   'genres', 
                                   'title', 
                                   'averageRating', 
                                   'numVotes', 
                                   'original_language', 
                                   'overview', 
                                   'poster_path'])[['primaryName_x', 'primaryName_y']].agg(lambda x: ', '.join(x.astype(str))).reset_index()

In [25]:
#Enlever les doublons dans les colonnes
# Fonction pour supprimer les doublons d'une liste tout en préservant l'ordre
def remove_duplicates_keep_order(lst):
    seen = set()
    return [x for x in lst if not (x in seen or seen.add(x))]

# Appliquer la fonction à chaque cellule des colonnes 'primaryName_x' et 'primaryName_y'
df_merged_7['primaryName_x'] = df_merged_7['primaryName_x'].apply(lambda x: ', '.join(remove_duplicates_keep_order(x.split(', '))))
df_merged_7['primaryName_y'] = df_merged_7['primaryName_y'].apply(lambda x: ', '.join(remove_duplicates_keep_order(x.split(', '))))

In [26]:
#Analyser le dataframe 
df_merged_7.head()

,tconst,startYear,genres,title,averageRating,numVotes,original_language,overview,poster_path,primaryName_x,primaryName_y
0,tt0035423,2001,"Comedy,Fantasy,Romance",Kate et Léopold,6.4,89260,en,When her scientist ex-boyfriend discovers a po...,/mUvikzKJJSg9khrVdxK8kg3TMHA.jpg,"Meg Ryan, Hugh Jackman, Liev Schreiber, Brecki...",James Mangold
1,tt0064177,1970,"Sci-Fi,Thriller",Le cerveau d'acier,7.1,10251,en,Forbin is the designer of an incredibly sophis...,/kmVDlTzERxSvBrqtQurjKf1zEy1.jpg,"Eric Braeden, Susan Clark, Gordon Pinsent, Wil...",Joseph Sargent
2,tt0065134,1970,"Adventure,Drama,Romance",Sierra torride,7.0,30734,en,When a wandering mercenary named Hogan rescues...,/2G9eM8UAM57TanLBhuPmz3jaNZG.jpg,"Clint Eastwood, Shirley MacLaine, Manolo Fábre...",Don Siegel
3,tt0065143,1970,"Horror,Mystery,Thriller",L'oiseau au plumage de cristal,7.1,24224,it,An American writer living in Rome witnesses an...,/dpALRkwodEIT7TGX9SqTgwVNn0i.jpg,"Tony Musante, Suzy Kendall, Enrico Maria Saler...",Dario Argento
4,tt0065377,1970,"Action,Drama,Thriller",Airport,6.6,21932,en,"Melodrama about a bomber on board an airplane,...",/6iOKluJfyiU1SLZbGYrY7Z0i6k0.jpg,"Burt Lancaster, Dean Martin, George Kennedy, J...",George Seaton


In [28]:
df_merged_7.shape

(10244, 11)

In [27]:
df_merged_7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10244 entries, 0 to 10243
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             10244 non-null  object 
 1   startYear          10244 non-null  int64  
 2   genres             10244 non-null  object 
 3   title              10244 non-null  object 
 4   averageRating      10244 non-null  float64
 5   numVotes           10244 non-null  int64  
 6   original_language  10244 non-null  object 
 7   overview           10244 non-null  object 
 8   poster_path        10244 non-null  object 
 9   primaryName_x      10244 non-null  object 
 10  primaryName_y      10244 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 880.5+ KB


In [28]:
df_merged_7.isna().sum()

tconst               0
startYear            0
genres               0
title                0
averageRating        0
numVotes             0
original_language    0
overview             0
poster_path          0
primaryName_x        0
primaryName_y        0
dtype: int64

In [29]:
#Créer dataframe recommandations
#Organiser les colonnes
df_merged_8 = df_merged_7[['tconst', 'title', 'startYear', 'original_language', 'primaryName_y', 'primaryName_x', 'averageRating', 'numVotes', 'genres', 'overview', 'poster_path']]
df_merged_8.head()

,tconst,title,startYear,original_language,primaryName_y,primaryName_x,averageRating,numVotes,genres,overview,poster_path
0,tt0035423,Kate et Léopold,2001,en,James Mangold,"Meg Ryan, Hugh Jackman, Liev Schreiber, Brecki...",6.4,89260,"Comedy,Fantasy,Romance",When her scientist ex-boyfriend discovers a po...,/mUvikzKJJSg9khrVdxK8kg3TMHA.jpg
1,tt0064177,Le cerveau d'acier,1970,en,Joseph Sargent,"Eric Braeden, Susan Clark, Gordon Pinsent, Wil...",7.1,10251,"Sci-Fi,Thriller",Forbin is the designer of an incredibly sophis...,/kmVDlTzERxSvBrqtQurjKf1zEy1.jpg
2,tt0065134,Sierra torride,1970,en,Don Siegel,"Clint Eastwood, Shirley MacLaine, Manolo Fábre...",7.0,30734,"Adventure,Drama,Romance",When a wandering mercenary named Hogan rescues...,/2G9eM8UAM57TanLBhuPmz3jaNZG.jpg
3,tt0065143,L'oiseau au plumage de cristal,1970,it,Dario Argento,"Tony Musante, Suzy Kendall, Enrico Maria Saler...",7.1,24224,"Horror,Mystery,Thriller",An American writer living in Rome witnesses an...,/dpALRkwodEIT7TGX9SqTgwVNn0i.jpg
4,tt0065377,Airport,1970,en,George Seaton,"Burt Lancaster, Dean Martin, George Kennedy, J...",6.6,21932,"Action,Drama,Thriller","Melodrama about a bomber on board an airplane,...",/6iOKluJfyiU1SLZbGYrY7Z0i6k0.jpg


In [30]:
#Renommer les colonnes
df_merge_9 = df_merged_8.rename(columns={'tconst': 'id', 
                                             'title': 'titre',
                                             'startYear' : 'annee',
                                             'original_language' : 'langue_originale',
                                             'primaryName_y' : 'realisateurs',
                                             'primaryName_x' : 'acteurs',
                                             'averageRating' : 'note',
                                             'numVotes' : 'nombre_votants',
                                             'genres' : 'genres', 
                                             'overview' : 'synopsis',
                                             'poster_path' : 'affiche'})
df_merge_9.head()

,id,titre,annee,langue_originale,realisateurs,acteurs,note,nombre_votants,genres,synopsis,affiche
0,tt0035423,Kate et Léopold,2001,en,James Mangold,"Meg Ryan, Hugh Jackman, Liev Schreiber, Brecki...",6.4,89260,"Comedy,Fantasy,Romance",When her scientist ex-boyfriend discovers a po...,/mUvikzKJJSg9khrVdxK8kg3TMHA.jpg
1,tt0064177,Le cerveau d'acier,1970,en,Joseph Sargent,"Eric Braeden, Susan Clark, Gordon Pinsent, Wil...",7.1,10251,"Sci-Fi,Thriller",Forbin is the designer of an incredibly sophis...,/kmVDlTzERxSvBrqtQurjKf1zEy1.jpg
2,tt0065134,Sierra torride,1970,en,Don Siegel,"Clint Eastwood, Shirley MacLaine, Manolo Fábre...",7.0,30734,"Adventure,Drama,Romance",When a wandering mercenary named Hogan rescues...,/2G9eM8UAM57TanLBhuPmz3jaNZG.jpg
3,tt0065143,L'oiseau au plumage de cristal,1970,it,Dario Argento,"Tony Musante, Suzy Kendall, Enrico Maria Saler...",7.1,24224,"Horror,Mystery,Thriller",An American writer living in Rome witnesses an...,/dpALRkwodEIT7TGX9SqTgwVNn0i.jpg
4,tt0065377,Airport,1970,en,George Seaton,"Burt Lancaster, Dean Martin, George Kennedy, J...",6.6,21932,"Action,Drama,Thriller","Melodrama about a bomber on board an airplane,...",/6iOKluJfyiU1SLZbGYrY7Z0i6k0.jpg


In [31]:
#Concatenner df_merge_9 et df_films_fr
df_recommandations = pd.concat([df_merge_9, df_films_fr])
df_recommandations.head()

,id,titre,annee,langue_originale,realisateurs,acteurs,note,nombre_votants,genres,synopsis,affiche,affiche\r
0,tt0035423,Kate et Léopold,2001,en,James Mangold,"Meg Ryan, Hugh Jackman, Liev Schreiber, Brecki...",6.4,89260,"Comedy,Fantasy,Romance",When her scientist ex-boyfriend discovers a po...,/mUvikzKJJSg9khrVdxK8kg3TMHA.jpg,NaN
1,tt0064177,Le cerveau d'acier,1970,en,Joseph Sargent,"Eric Braeden, Susan Clark, Gordon Pinsent, Wil...",7.1,10251,"Sci-Fi,Thriller",Forbin is the designer of an incredibly sophis...,/kmVDlTzERxSvBrqtQurjKf1zEy1.jpg,NaN
2,tt0065134,Sierra torride,1970,en,Don Siegel,"Clint Eastwood, Shirley MacLaine, Manolo Fábre...",7.0,30734,"Adventure,Drama,Romance",When a wandering mercenary named Hogan rescues...,/2G9eM8UAM57TanLBhuPmz3jaNZG.jpg,NaN
3,tt0065143,L'oiseau au plumage de cristal,1970,it,Dario Argento,"Tony Musante, Suzy Kendall, Enrico Maria Saler...",7.1,24224,"Horror,Mystery,Thriller",An American writer living in Rome witnesses an...,/dpALRkwodEIT7TGX9SqTgwVNn0i.jpg,NaN
4,tt0065377,Airport,1970,en,George Seaton,"Burt Lancaster, Dean Martin, George Kennedy, J...",6.6,21932,"Action,Drama,Thriller","Melodrama about a bomber on board an airplane,...",/6iOKluJfyiU1SLZbGYrY7Z0i6k0.jpg,NaN


In [34]:
df_recommandations.shape

(17079, 12)

In [32]:
df_recommandations.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17079 entries, 0 to 6834
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                17079 non-null  object 
 1   titre             17079 non-null  object 
 2   annee             17079 non-null  int64  
 3   langue_originale  17079 non-null  object 
 4   realisateurs      17079 non-null  object 
 5   acteurs           17079 non-null  object 
 6   note              17079 non-null  float64
 7   nombre_votants    17079 non-null  int64  
 8   genres            17079 non-null  object 
 9   synopsis          17079 non-null  object 
 10  affiche           10244 non-null  object 
          6835 non-null   object 
dtypes: float64(1), int64(2), object(9)
memory usage: 1.7+ MB


In [33]:
df_recommandations.isna().sum()

id                      0
titre                   0
annee                   0
langue_originale        0
realisateurs            0
acteurs                 0
note                    0
nombre_votants          0
genres                  0
synopsis                0
affiche              6835
affiche\r           10244
dtype: int64

In [34]:
# enlever les doubles
df_recommandations_nondouble = df_recommandations.drop_duplicates(subset='id', keep='first')
print(df_recommandations_nondouble)

             id                           titre  annee langue_originale  \
0     tt0035423                 Kate et Léopold   2001               en   
1     tt0064177              Le cerveau d'acier   1970               en   
2     tt0065134                  Sierra torride   1970               en   
3     tt0065143  L'oiseau au plumage de cristal   1970               it   
4     tt0065377                         Airport   1970               en   
...         ...                             ...    ...              ...   
6830  tt9861288                        Boutchou   2020               fr   
6831  tt9866344                   Mon ami Walid   2019               fr   
6832  tt9866700        Paranormal Investigation   2018               fr   
6833  tt9894450                        Felicità   2020               fr   
6834  tt9908390                         Le Lion   2020               fr   

                realisateurs  \
0              James Mangold   
1             Joseph Sargent   
2  

In [35]:
#Exporter dataframe recommandations en csv
df_recommandations_nondouble.to_csv("data_recommandations.csv", sep = "\t", encoding = "utf-8", index = False)